In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

h1b_TRAIN_tableau.csv



# Data import

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import time, sleep
from tqdm import tqdm, trange
from tqdm import tqdm_notebook as tn
tqdm.pandas(desc='progress bar')
from collections import Counter
import operator
import pickle
import re
import math
import os
get_ipython().magic('matplotlib inline')



from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.cross_validation import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
df = pd.read_csv('../input/h1b_TRAIN_tableau.csv', sep=',')

In [5]:
df.describe()

,Case Id,JT_EXECUTIVE_FLAG,Prevailing Wage filter,Longitude,Latitude,SoC_COMPUTER_FLAG,SoC_UPPERCASE_FLAG,JT_PROFESSOR_FLAG,Number of Records,Prevailing Wage,JT_SENIOR_FLAG,JT_DATASCIENCE_FLAG,Emp_UNIVERSITY_FLAG,Year
count,2.251844e+06,2.251844e+06,2.251844e+06,2.171398e+06,2.171398e+06,2.251844e+06,2.251844e+06,2.251844e+06,2251844.0,2.251779e+06,2.251844e+06,2.251844e+06,2.251844e+06,2.251836e+06
mean,1.501788e+06,2.492624e-03,9.964092e-01,9.213684e+01,3.815967e+01,6.126370e-01,4.274483e-01,6.115210e-02,1.0,1.498183e+05,9.246999e-02,4.810280e-03,5.575031e-02,2.013854e+03
std,8.666892e+05,4.986393e-02,5.981590e-02,1.963492e+01,4.671896e+00,4.871478e-01,4.947084e-01,2.396092e-01,0.0,5.808395e+06,2.896883e-01,6.918918e-02,2.294390e-01,1.680675e+00
min,1.000000e+00,0.000000e+00,0.000000e+00,-1.457298e+02,1.343719e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.011000e+03
25%,7.512768e+05,0.000000e+00,1.000000e+00,7.551381e+01,3.416536e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.0,5.439200e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.012000e+03
50%,1.502439e+06,0.000000e+00,1.000000e+00,8.623793e+01,3.910312e+01,1.000000e+00,0.000000e+00,0.000000e+00,1.0,6.502100e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.014000e+03
75%,2.252488e+06,0.000000e+00,1.000000e+00,1.119261e+02,4.088399e+01,1.000000e+00,1.000000e+00,0.000000e+00,1.0,8.143200e+04,0.000000e+00,0.000000e+00,0.000000e+00,2.015000e+03
max,3.002457e+06,1.000000e+00,1.000000e+00,1.578583e+02,6.483778e+01,1.000000e+00,1.000000e+00,1.000000e+00,1.0,6.997607e+09,1.000000e+00,1.000000e+00,1.000000e+00,2.016000e+03


In [6]:
df.shape

(2251844, 22)

In [7]:
df.head(10)

,Case Id,Case Status,JT_EXECUTIVE_FLAG,Prevailing Wage filter,Longitude,Latitude,SoC_COMPUTER_FLAG,Employer Name,SoC_UPPERCASE_FLAG,JT_PROFESSOR_FLAG,...,Number of Records,Prevailing Wage,JT_SENIOR_FLAG,JT_DATASCIENCE_FLAG,Emp_UNIVERSITY_FLAG,Soc Name,Worksite - Split 1,Worksite - Split 2,COAST_FLAG,Year
0,2817253,CERTIFIED,0,1,74.222116,40.417095,0,YANG'S SEAFOOD LLC,0,0,...,1,52852.8,0,0,0,Accountants,ABERDEEN,NEW JERSEY,"CALIFORNIA, MASSACHUSETTS, NEW JERSEY and 2 more",2011.0
1,2854337,CERTIFIED,0,1,96.829169,32.961790,0,"CRYSTAL TECHNOLOGY & INDUSTRIES, INC.",0,0,...,1,41600.0,0,0,0,Accountants,ADDISON,TEXAS,Other,2011.0
2,2662898,CERTIFIED,0,1,77.046921,38.804835,0,FIRST PAINTING INC.,0,0,...,1,48963.2,0,0,0,Accountants,ALEXANDRIA,VIRGINIA,Other,2011.0
3,2827634,CERTIFIED,0,1,118.127015,34.095287,0,"YANG AND COMPANY CPA, A PROFESSIONAL CORP.",0,0,...,1,41558.4,0,0,0,Accountants,ALHAMBRA,CALIFORNIA,"CALIFORNIA, MASSACHUSETTS, NEW JERSEY and 2 more",2011.0
4,2805991,CERTIFIED,0,1,118.127015,34.095287,0,"YANG AND COMPANY CPA, A PROFESSIONAL CORPORATION",0,0,...,1,41558.4,0,0,0,Accountants,ALHAMBRA,CALIFORNIA,"CALIFORNIA, MASSACHUSETTS, NEW JERSEY and 2 more",2011.0
5,2885264,CERTIFIED,0,1,75.490183,40.608430,0,PANDYA CONSULTING LLC,0,0,...,1,42473.6,0,0,0,Accountants,ALLENTOWN,PENNSYLVANIA,Other,2011.0
6,2908732,CERTIFIED,0,1,93.631913,42.030781,0,IOWA STATE UNIVERSITY OF SCIENCE AND TECHNOLOGY,0,0,...,1,38500.8,0,0,1,Accountants,AMES,IOWA,Other,2011.0
7,2692448,CERTIFIED,0,1,117.914504,33.835293,0,NIDI TEC INC.,0,0,...,1,45198.4,0,0,0,Accountants,ANAHEIM,CALIFORNIA,"CALIFORNIA, MASSACHUSETTS, NEW JERSEY and 2 more",2011.0
8,2676280,CERTIFIED,0,1,117.914504,33.835293,0,ROBERT CHANG ACCOUNTANCY CORP.,0,0,...,1,45198.4,0,0,0,Accountants,ANAHEIM,CALIFORNIA,"CALIFORNIA, MASSACHUSETTS, NEW JERSEY and 2 more",2011.0
9,2903958,CERTIFIED,0,1,117.914504,33.835293,0,VINEERAAJ INC.,0,0,...,1,45198.4,0,0,0,Accountants,ANAHEIM,CALIFORNIA,"CALIFORNIA, MASSACHUSETTS, NEW JERSEY and 2 more",2011.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2251844 entries, 0 to 2251843
Data columns (total 22 columns):
Case Id                   int64
Case Status               object
JT_EXECUTIVE_FLAG         int64
Prevailing Wage filter    int64
Longitude                 float64
Latitude                  float64
SoC_COMPUTER_FLAG         int64
Employer Name             object
SoC_UPPERCASE_FLAG        int64
JT_PROFESSOR_FLAG         int64
Full Time Position        object
Job Title                 object
Number of Records         int64
Prevailing Wage           float64
JT_SENIOR_FLAG            int64
JT_DATASCIENCE_FLAG       int64
Emp_UNIVERSITY_FLAG       int64
Soc Name                  object
Worksite - Split 1        object
Worksite - Split 2        object
COAST_FLAG                object
Year                      float64
dtypes: float64(4), int64(10), object(8)
memory usage: 378.0+ MB


In [9]:
df2=df.copy()

In [10]:
df2['Case Status'].unique()

array(['CERTIFIED', 'CERTIFIED-WITHDRAWN', 'DENIED', 'WITHDRAWN',
       'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED', nan,
       'INVALIDATED', 'REJECTED'], dtype=object)

In [11]:
df3 = df2[(df2['Case Status'].isin(['CERTIFIED', 'CERTIFIED-WITHDRAWN', 'WITHDRAWN', 'DENIED']))&(df2['Prevailing Wage'] < 1000000000)&(df2['Prevailing Wage'] > -1)]
df3.shape

(2251768, 22)

In [12]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2251768 entries, 0 to 2251843
Data columns (total 22 columns):
Case Id                   int64
Case Status               object
JT_EXECUTIVE_FLAG         int64
Prevailing Wage filter    int64
Longitude                 float64
Latitude                  float64
SoC_COMPUTER_FLAG         int64
Employer Name             object
SoC_UPPERCASE_FLAG        int64
JT_PROFESSOR_FLAG         int64
Full Time Position        object
Job Title                 object
Number of Records         int64
Prevailing Wage           float64
JT_SENIOR_FLAG            int64
JT_DATASCIENCE_FLAG       int64
Emp_UNIVERSITY_FLAG       int64
Soc Name                  object
Worksite - Split 1        object
Worksite - Split 2        object
COAST_FLAG                object
Year                      float64
dtypes: float64(4), int64(10), object(8)
memory usage: 395.1+ MB


In [13]:
#pd.isnull(df3).sum()
df3['Latitude'].fillna(0,inplace = True)
df3['Longitude'].fillna(0,inplace = True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [14]:
df3['Year'] = df3['Year'].astype(float)
df3['Year'] = df3['Year'].astype(int)
df3.Year = df3.Year.astype(str)

df3['Longitude'] = df3['Longitude'].astype(float)
df3['Longitude'] = df3['Longitude'].astype(int)
df3.Longitude = df3.Longitude.astype(str)

df3['Latitude'] = df3['Latitude'].astype(float)
df3['Latitude'] = df3['Latitude'].astype(int)
df3.Latitude = df3.Latitude.astype(str)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.o

In [15]:
df4 = df3.fillna('BLANK')

In [16]:
pd.isnull(df4).sum()

Case Id                   0
Case Status               0
JT_EXECUTIVE_FLAG         0
Prevailing Wage filter    0
Longitude                 0
Latitude                  0
SoC_COMPUTER_FLAG         0
Employer Name             0
SoC_UPPERCASE_FLAG        0
JT_PROFESSOR_FLAG         0
Full Time Position        0
Job Title                 0
Number of Records         0
Prevailing Wage           0
JT_SENIOR_FLAG            0
JT_DATASCIENCE_FLAG       0
Emp_UNIVERSITY_FLAG       0
Soc Name                  0
Worksite - Split 1        0
Worksite - Split 2        0
COAST_FLAG                0
Year                      0
dtype: int64

In [18]:
df4[pd.isnull(df4.SOC_MOD)]

AttributeError: 'DataFrame' object has no attribute 'SOC_MOD'

In [17]:
del df4['Prevailing Wage filter']
del df4['Number of Records']
#del df4['Longitude']
#del df4['Latitude']

In [47]:
df5 = df4.copy()

In [21]:
# pickle.dump(df5, open('df5.pkl', 'wb'))
# df5 = pickle.load(open('df5.pkl', 'rb'))

OSError: [Errno 28] No space left on device

# Feature Engineering
*     Soc Name
*     Employer Name
*     Job Title

# 1. Cleaning text data

In [48]:
import string
from time import time
t0 = time()
table = {ord(char):ord(' ') if char!= '&' else ord(char) for char in string.punctuation}
table_digits  =  {ord(char):ord(' ') for char in string.digits}
def remove_punctuation_and_casing(line):
    line = line.translate(table)
    line = line.translate(table_digits)
    line = line.upper()
    return line
df5['Soc Name'].fillna('BLANK',inplace=True)
df5['SOC_MOD'] = df5['Soc Name'].progress_apply(remove_punctuation_and_casing)
print (df5['SOC_MOD'].sample(2))

df5['Job Title'].fillna('BLANK',inplace=True)
df5['JT_MOD'] = df5['Job Title'].progress_apply(remove_punctuation_and_casing)
print (df5['JT_MOD'].sample(2))

df5['Employer Name'].fillna('BLANK',inplace=True)
df5['EMP_MOD'] = df5['Employer Name'].progress_apply(remove_punctuation_and_casing)
print (df5['EMP_MOD'].sample(2))

progress bar:   0%|          | 0/2251768 [00:00<?, ?it/s]

1502125    BIOCHEMISTS AND BIOPHYSICISTS
1582322                     LOGISTICIANS
Name: SOC_MOD, dtype: object


progress bar:   1%|          | 15476/2251768 [00:00<00:14, 154757.20it/s]

386024                             BUSINESS ANALYST
1743632    COMPUTER SOFTWARE ENGINEER  APPLICATIONS
Name: JT_MOD, dtype: object


progress bar: 100%|██████████| 2251768/2251768 [00:07<00:00, 290164.88it/s]


1638586    ARUP NORTH AMERICA LTD 
82634          YELLOWPAGES COM LLC
Name: EMP_MOD, dtype: object


In [49]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from nltk.stem import PorterStemmer
try:
    stemmed = PorterStemmer().stem(word_tokenize('I AM THE JACK OF ALL TRADES')[6])
    print (word_tokenize("I AM SAM"))
    print (stemmed)
except:
    print ('IMPORT FAILED. CHECK NLTK DATA')

['I', 'AM', 'SAM']
trade


In [50]:
t0= time()
stemmer = PorterStemmer()
def stem_line(line):
    line = line.split()
    line = [stemmer.stem(word) for word in line]
    line = " ".join(line)
    line = line.upper()
    return line
df5['SOC_MOD'] = df5['SOC_MOD'].progress_apply(stem_line)
t1 = time()
print ("elapsed time is {} seconds".format(int(t1-t0)))
print (df5['SOC_MOD'].sample(2))

df5['JT_MOD'] = df5['JT_MOD'].progress_apply(stem_line)
t1 = time()
print ("elapsed time is {} seconds".format(int(t1-t0)))
print (df5['JT_MOD'].sample(2))

df5['EMP_MOD'] = df5['EMP_MOD'].progress_apply(stem_line)
t1 = time()
print ("elapsed time is {} seconds".format(int(t1-t0)))
print (df5['EMP_MOD'].sample(2))

progress bar:   0%|          | 1530/2251768 [00:00<02:27, 15294.84it/s]

elapsed time is 145 seconds
1177667    COMPUT SOFTWAR ENGIN APPLIC
422056           COMPUT SYSTEM ANALYST
Name: SOC_MOD, dtype: object


progress bar:   0%|          | 0/2251768 [00:00<?, ?it/s]

elapsed time is 268 seconds
739495                 CONSULT
1726469    SR PROGRAMM ANALYST
Name: JT_MOD, dtype: object


progress bar: 100%|██████████| 2251768/2251768 [02:28<00:00, 15211.25it/s]


elapsed time is 416 seconds
87640              DONGGUK UNIVERS LO ANGEL
961328    TWIN CITI YOUTH SOCCER ASSOCI INC
Name: EMP_MOD, dtype: object


# 2. NMF and LDA for Topic modelling on SoC, JT, and EMP

# SOC NAME

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [52]:
documents_SOC = df5['SOC_MOD']
no_features = 10000
no_topics = 18
no_top_words = 10

In [53]:
documents_SOC.shape

(2251768,)

In [54]:
# NMF is able to use tf-idf
tfidf_vectorizer_SOC = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf_SOC = tfidf_vectorizer_SOC.fit_transform(documents_SOC)
tfidf_feature_names_SOC = tfidf_vectorizer_SOC.get_feature_names()

In [55]:
# Run NMF
nmf_SOC = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_SOC)
display_topics(nmf_SOC, tfidf_feature_names_SOC, no_top_words)

Topic 0:
analyst comput network secur credit budget non softwar applic web
Topic 1:
applic softwar develop web comput engin non train secur specialist
Topic 2:
programm comput inform network non specialist softwar applic support architect
Topic 3:
occup comput inform therapist network specialist softwar applic support architect
Topic 4:
engin electron comput industri civil sale softwar applic chemic hardwar
Topic 5:
manag inform analyst sale gener architectur servic market purchas health
Topic 6:
administr network databas comput educ architect inform servic support specialist
Topic 7:
account auditor bookkeep audit clerk collector zoologist estat execut excav
Topic 8:
market specialist research analyst public support relat train resourc human
Topic 9:
financi analyst specialist manag secur commod person advisor sa examin
Topic 10:
softwar develop assur qualiti tester engin comput non amp escort
Topic 11:
scientist epidemiologist medic biolog inform materi technician technologist life r

In [56]:
pickle.dump(nmf_SOC, open('nmf_SOC.pkl', 'wb'))

OSError: [Errno 28] No space left on device

In [192]:
nmf_SOC = pickle.load(open('nmf_SOC.pkl', 'rb'))

Topic 0:
analyst comput budget research credit electron network hardwar secur non
Topic 1:
programm comput inform non electron network hardwar support engin specialist
Topic 2:
applic softwar develop train web comput engin non specialist secur
Topic 3:
occup comput inform therapist electron network hardwar support specialist scienc
Topic 4:
account auditor audit bookkeep clerk flight equip examin event ethnic
Topic 5:
manag inform analyst architectur servic gener sale health market promot
Topic 6:
engin mechan civil electr electron industri chemic architectur biomed hardwar
Topic 7:
market specialist research analyst public relat support resourc human train
Topic 8:
administr network databas comput architect educ support servic specialist inform
Topic 9:
scientist biolog epidemiologist medic technician inform materi research technologist food
Topic 10:
financi analyst specialist manag person advisor commod secur examin budget
Topic 11:
teacher postsecondari scienc busi special school s

In [193]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
# tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
# tf = tf_vectorizer.fit_transform(documents)
# tf_feature_names = tf_vectorizer.get_feature_names()

In [194]:
# Run LDA
# lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5,n_jobs=4 ,learning_method='online',verbose=2,learning_offset=50.,random_state=0, batch_size=10000).fit(tf)
# display_topics(lda, tf_feature_names, no_top_words)

In [57]:
nmf_topics_SOC = nmf_SOC.transform(tfidf_SOC)
best_topic_SOC = np.argmax(nmf_topics_SOC,axis=1)
d_SOC = pd.DataFrame(data=best_topic_SOC, columns = ['topic_SOC'])

In [58]:
!free -g

             total       used       free     shared    buffers     cached
Mem:           113         82         30          1          0         56
-/+ buffers/cache:         25         87
Swap:            0          0          0


In [59]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay        1007G   46G  962G   5% /
tmpfs            64M     0   64M   0% /dev
tmpfs            57G     0   57G   0% /sys/fs/cgroup
/dev/loop1      976M  976M     0 100% /tmp
/dev/md127      1.5T  478G  998G  33% /kaggle/input
/dev/sdb       1007G   46G  962G   5% /etc/hosts
shm              64M  4.0K   64M   1% /dev/shm
/dev/sda1        99G  4.0G   91G   5% /kaggle/lib/kagglegym
tmpfs            57G     0   57G   0% /sys/firmware


# ** Job Title**

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [61]:
documents_JT = df5['JT_MOD']
no_features = 10000
no_topics = 18
no_top_words = 10

In [62]:
# NMF is able to use tf-idf
tfidf_vectorizer_JT = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf_JT = tfidf_vectorizer_JT.fit_transform(documents_JT)
tfidf_feature_names_JT = tfidf_vectorizer_JT.get_feature_names()

In [63]:
# Run NMF
nmf_JT = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_JT)
display_topics(nmf_JT, tfidf_feature_names_JT, no_top_words)

Topic 0:
programm analyst configur level sr sa java statist ii profession
Topic 1:
softwar engin tester develop architect member yahoo mt embed dev
Topic 2:
comput configur specialist architect network support admin inform qualiti scientist
Topic 3:
develop java web net sr ii interfac user etl specialist
Topic 4:
consult technic sap princip er lead function sr manag infrastructur
Topic 5:
analyst technolog financi qa market ii sr data jc sap
Topic 6:
manag project product program technic market account oper advisori jc
Topic 7:
technolog lead technic architect team inform modul project solut practition
Topic 8:
senior advisori jc architect specialist tax design scientist account data
Topic 9:
associ postdoctor advisori director research attorney architect professor doctor post
Topic 10:
busi intellig specialist oper director process sr manag analyst intern
Topic 11:
engin design mechan sr staff ii network process lead product
Topic 12:
assist professor clinic visit presid vice audit me

In [201]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
# tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
# tf = tf_vectorizer.fit_transform(documents)
# tf_feature_names = tf_vectorizer.get_feature_names()

In [202]:
# Run LDA
# lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5,n_jobs=4 ,learning_method='online',verbose=2,learning_offset=50.,random_state=0, batch_size=10000).fit(tf)
# display_topics(lda, tf_feature_names, no_top_words)

In [64]:
nmf_topics_JT = nmf_JT.transform(tfidf_JT)
best_topic_JT = np.argmax(nmf_topics_JT,axis=1)
d_JT = pd.DataFrame(data=best_topic_JT, columns = ['topic_JT'])

# ** Employer Name**

In [140]:
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.decomposition import NMF, LatentDirichletAllocation

# def display_topics(model, feature_names, no_top_words):
#     for topic_idx, topic in enumerate(model.components_):
#         print ("Topic %d:" % (topic_idx))
#         print (" ".join([feature_names[i]
#                         for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [141]:
# documents_EMP = df5['EMP_MOD']
# no_features = 10000
# no_topics = 18
# no_top_words = 10

In [142]:
# # NMF is able to use tf-idf
# tfidf_vectorizer_EMP = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
# tfidf_EMP = tfidf_vectorizer_EMP.fit_transform(documents_EMP)
# tfidf_feature_names_EMP = tfidf_vectorizer_EMP.get_feature_names()

In [143]:
# # Run NMF
# nmf_EMP = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf_EMP)
# display_topics(nmf_EMP, tfidf_feature_names_EMP, no_top_words)

In [144]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
# tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
# tf = tf_vectorizer.fit_transform(documents)
# tf_feature_names = tf_vectorizer.get_feature_names()

In [145]:
# Run LDA
# lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5,n_jobs=4 ,learning_method='online',verbose=2,learning_offset=50.,random_state=0, batch_size=10000).fit(tf)
# display_topics(lda, tf_feature_names, no_top_words)

In [146]:
# nmf_topics_EMP = nmf_EMP.transform(tfidf_EMP)
# best_topic_EMP = np.argmax(nmf_topics_EMP,axis=1)
# d_EMP = pd.DataFrame(data=best_topic_EMP, columns = ['topic_EMP'])

# Other flags and dummies

In [65]:
other_dummies = pd.DataFrame()
other_dummies['JT_EXECUTIVE_FLAG'] = df5['JT_EXECUTIVE_FLAG']
other_dummies['SoC_COMPUTER_FLAG'] = df5['SoC_COMPUTER_FLAG']
other_dummies['SoC_UPPERCASE_FLAG'] = df5['SoC_UPPERCASE_FLAG']
other_dummies['JT_PROFESSOR_FLAG'] = df5['JT_PROFESSOR_FLAG']
other_dummies['JT_SENIOR_FLAG'] = df5['JT_SENIOR_FLAG']
other_dummies['JT_DATASCIENCE_FLAG'] = df5['JT_DATASCIENCE_FLAG']
other_dummies['Emp_UNIVERSITY_FLAG'] = df5['Emp_UNIVERSITY_FLAG']

In [66]:
df5['City_State'] = df5[['Latitude', 'Longitude']].apply(lambda x: ' : '.join(x), axis=1)
city_dummies = pd.get_dummies(df5['City_State'],drop_first=True)
# city_dummies.head(2)

In [67]:
coast_dummies = pd.get_dummies(df5['COAST_FLAG'],drop_first=True)
#coast_dummies.head(2)

In [102]:
year_dummies = pd.get_dummies(df5['Year'],drop_first=True)
#coast_dummies.head(2)

In [68]:
state_dummies = pd.get_dummies(df5['Worksite - Split 2'],drop_first=True)
#state_dummies.head(2)

In [69]:
fullpart_dummies = pd.get_dummies(df5['Full Time Position'],drop_first=True)
# fullpart_dummies.head(2)

In [70]:
#df5['Prevailing Wage'].max()
df5['Wage_buckets'] = pd.cut(df5['Prevailing Wage'],[-1,10000,30000,50000,70000,90000,110000,130000,150000,170000,190000,10000000000000])
#df5.head()
wage_dummies = pd.get_dummies(df5['Wage_buckets'],drop_first=True)
#wage_dummies.head(2)

In [71]:
SOC_nmf_dummies = pd.get_dummies(d_SOC,drop_first=True)
JT_nmf_dummies = pd.get_dummies(d_JT,drop_first=True)
# EMP_nmf_dummies = pd.get_dummies(d_EMP,drop_first=True)

In [103]:
other_dummies.shape
fullpart_dummies.shape
state_dummies.shape
year_dummies.shape
coast_dummies.shape
wage_dummies.shape
city_dummies.shape
d_SOC.shape
d_JT.shape
# d_EMP.shape

(2251768, 1)

# Model

In [104]:
from scipy.sparse import hstack
X = hstack((tf_SOC,d_JT.as_matrix(),fullpart_dummies.as_matrix(),year_dummies.as_matrix(),city_dummies.as_matrix(),other_dummies.as_matrix(),state_dummies.as_matrix(),
            wage_dummies.as_matrix()))
y = df5['Case Status']

In [31]:
df6 = pd.concat([df5, city_dummies,other_dummies,state_dummies,fullpart_dummies,wage_dummies,coast_dummies], axis=1)

In [86]:
vectorizer1 = CountVectorizer(binary= False, stop_words='english', max_df = 0.5, min_df = 10, max_features = 200)
analy1 = vectorizer1.build_analyzer()
vectorizer2 = CountVectorizer(binary= False, stop_words='english', max_df = 0.5, min_df = 10, max_features = 200)
analy2 = vectorizer2.build_analyzer()
# vectorizer3 = CountVectorizer(binary= False, stop_words='english', max_df = 0.5, min_df = 10, max_features = 200)
# analy3 = vectorizer3.build_analyzer()

In [33]:
ctr1_SOC = Counter()
for name in tqdm(df5[df5['Case Status'] == 'CERTIFIED']['SOC_MOD']):
    ctr1_SOC.update(analy1(name))

ctr2_SOC = Counter()
for name in tqdm(df5[df5['Case Status'] == 'CERTIFIED-WITHDRAWN']['SOC_MOD']):
    ctr2_SOC.update(analy1(name))

ctr3_SOC = Counter()
for name in tqdm(df5[df5['Case Status'] == 'DENIED']['SOC_MOD']):
    ctr3_SOC.update(analy1(name))

ctr4_SOC = Counter()
for name in tqdm(df5[df5['Case Status'] == 'WITHDRAWN']['SOC_MOD']):
    ctr4_SOC.update(analy1(name))

100%|██████████| 67189/67189 [00:00<00:00, 162260.30it/s]


In [34]:
ctr1_JT = Counter()
for name in tqdm(df5[df5['Case Status'] == 'CERTIFIED']['JT_MOD']):
    ctr1_JT.update(analy2(name))

ctr2_JT = Counter()
for name in tqdm(df5[df5['Case Status'] == 'CERTIFIED-WITHDRAWN']['JT_MOD']):
    ctr2_JT.update(analy2(name))

ctr3_JT = Counter()
for name in tqdm(df5[df5['Case Status'] == 'DENIED']['JT_MOD']):
    ctr3_JT.update(analy2(name))

ctr4_JT = Counter()
for name in tqdm(df5[df5['Case Status'] == 'WITHDRAWN']['JT_MOD']):
    ctr4_JT.update(analy2(name))

100%|██████████| 67189/67189 [00:00<00:00, 151013.32it/s]


In [35]:
# ctr1_EMP = Counter()
# for name in tqdm(df5[df5['Case Status'] == 'CERTIFIED']['EMP_MOD']):
#     ctr1_EMP.update(analy3(name))

# ctr2_EMP = Counter()
# for name in tqdm(df5[df5['Case Status'] == 'CERTIFIED-WITHDRAWN']['EMP_MOD']):
#     ctr2_EMP.update(analy3(name))

# ctr3_EMP = Counter()
# for name in tqdm(df5[df5['Case Status'] == 'DENIED']['EMP_MOD']):
#     ctr3_EMP.update(analy3(name))

# ctr4_EMP = Counter()
# for name in tqdm(df5[df5['Case Status'] == 'WITHDRAWN']['EMP_MOD']):
#     ctr4_EMP.update(analy3(name))

In [36]:
SOC_terms_0 = [term[0] for term in ctr1_SOC.most_common(200)]
add_SOC_row0 = ' '.join(SOC_terms_0)
SOC_terms_1 = [term[0] for term in ctr2_SOC.most_common(10000) if (term[0] not in SOC_terms_0)]
add_SOC_row1 = ' '.join(SOC_terms_1)
SOC_terms_2 = [term[0] for term in ctr3_SOC.most_common(50000) if ((term[0] not in SOC_terms_0) & (term[0] not in SOC_terms_1))]
add_SOC_row2 = ' '.join(SOC_terms_2)
SOC_terms_3 = [term[0] for term in ctr4_SOC.most_common(50000) if ((term[0] not in SOC_terms_0) & (term[0] not in SOC_terms_1) & (term[0] not in SOC_terms_2))]
add_SOC_row3 = ' '.join(SOC_terms_3)

In [37]:
JT_terms_0 = [term[0] for term in ctr1_JT.most_common(200)]
add_JT_row0 = ' '.join(JT_terms_0)
JT_terms_1 = [term[0] for term in ctr2_JT.most_common(10000) if (term[0] not in JT_terms_0)]
add_JT_row1 = ' '.join(JT_terms_1)
JT_terms_2 = [term[0] for term in ctr3_JT.most_common(50000) if ((term[0] not in JT_terms_0) & (term[0] not in JT_terms_1))]
add_JT_row2 = ' '.join(JT_terms_2)
JT_terms_3 = [term[0] for term in ctr4_JT.most_common(50000) if ((term[0] not in JT_terms_0) & (term[0] not in JT_terms_1) & (term[0] not in JT_terms_2))]
add_JT_row3 = ' '.join(JT_terms_3)

In [38]:
# EMP_terms_0 = [term[0] for term in ctr1_EMP.most_common(200)]
# add_EMP_row0 = ' '.join(EMP_terms_0)
# EMP_terms_1 = [term[0] for term in ctr2_EMP.most_common(10000) if (term[0] not in EMP_terms_0)]
# add_EMP_row1 = ' '.join(EMP_terms_1)
# EMP_terms_2 = [term[0] for term in ctr3_EMP.most_common(50000) if ((term[0] not in EMP_terms_0) & (term[0] not in EMP_terms_1))]
# add_EMP_row2 = ' '.join(EMP_terms_2)
# EMP_terms_3 = [term[0] for term in ctr4_EMP.most_common(50000) if ((term[0] not in EMP_terms_0) & (term[0] not in EMP_terms_1) & (term[0] not in EMP_terms_2))]
# add_EMP_row3 = ' '.join(EMP_terms_3)

In [39]:
df6_CW = df6[df6['Case Status'] == 'CERTIFIED-WITHDRAWN']
df6_D = df6[df6['Case Status'] == 'DENIED']
df6_W = df6[df6['Case Status'] == 'WITHDRAWN']

In [ ]:
df6_CW['SOC_MOD'] = add_SOC_row1
df6_D['SOC_MOD'] = add_SOC_row2
df6_W['SOC_MOD'] = add_SOC_row3

df6_CW['JT_MOD'] = add_JT_row1
df6_D['JT_MOD'] = add_JT_row2
df6_W['JT_MOD'] = add_JT_row3

# df6_CW['EMP_MOD'] = add_EMP_row1
# df6_D['EMP_MOD'] = add_EMP_row2
# df6_W['EMP_MOD'] = add_EMP_row3

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df7 = df6.append([df6_CW]*0,ignore_index=True)
df8 = df7.append([df6_D]*1,ignore_index=True)
df9 = df8.append([df6_W]*1,ignore_index=True)

In [166]:
print (df9.shape)
df10 = df9[~pd.isnull(df9.SOC_MOD)]
df10.shape

(160999, 515)


(160996, 515)

In [167]:
# pickle.dump(df10, open('df10.pkl', 'wb'))

# df10 = pickle.load(open('df10.pkl', 'rb'))

OSError: [Errno 28] No space left on device

In [87]:
t0 = time()
tf_SOC = vectorizer1.fit_transform(df5['SOC_MOD'])
t1 = time()
print ("elapsed time is {} seconds for vectorization".format(int(t1-t0)))
tf_feature_names1 = vectorizer1.get_feature_names()
print (tf_feature_names1[0:2])
print ("NUMBER OF FEATURES",len(tf_feature_names1))

elapsed time is 10 seconds for vectorization
['account', 'actuari']
NUMBER OF FEATURES 200


In [88]:
t0 = time()
tf_JT = vectorizer2.fit_transform(df5['JT_MOD'])
t1 = time()
print ("elapsed time is {} seconds for vectorization".format(int(t1-t0)))
tf_feature_names2 = vectorizer2.get_feature_names()
print (tf_feature_names2[0:2])
print ("NUMBER OF FEATURES",len(tf_feature_names2))

elapsed time is 10 seconds for vectorization
['account', 'actuari']
NUMBER OF FEATURES 200


In [ ]:
# t0 = time()
# tf_EMP = vectorizer3.fit_transform(df10['EMP_MOD'])
# t1 = time()
# print ("elapsed time is {} seconds for vectorization".format(int(t1-t0)))
# tf_feature_names3 = vectorizer3.get_feature_names()
# print (tf_feature_names3[0:2])
# print ("NUMBER OF FEATURES",len(tf_feature_names3))

In [170]:
X = df10.iloc[:,27:]
y = df10['Case Status']

In [171]:
rest_all.head()

,13 : -144,15 : -145,17 : 64,18 : 66,18 : 67,19 : 155,21 : 157,25 : 80,25 : 97,26 : 80,...,"(30000, 50000]","(50000, 70000]","(70000, 90000]","(90000, 110000]","(110000, 130000]","(130000, 150000]","(150000, 170000]","(170000, 190000]","(190000, 10000000000000]",Other
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [172]:
# from scipy.sparse import hstack
# X = hstack(tf_SOC,tf_JT,rest_all.as_matrix())
# y = df5['Case Status']

In [173]:
y.shape

(160996,)

In [105]:
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size = 0.3,random_state = 100)

print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

labels = y_train.value_counts().reset_index()
labels[labels['index'] == 'CERTIFIED']['Case Status'].values[0]
print (labels)

(1576237, 721) (675531, 721) (1576237,) (675531,)
                 index  Case Status
0            CERTIFIED      1373218
1  CERTIFIED-WITHDRAWN       106531
2               DENIED        49493
3            WITHDRAWN        46995


In [175]:
#print (y_train[0:10])
# weight_list[0:10]

In [114]:
clf = MultinomialNB(alpha=0.05)

#     LogisticRegressionCV() - 91
#     MultinomialNB(alpha=1.0) - 65
#     PassiveAggressiveClassifier() - 89
#     KNeighborsClassifier(3) - Fail
#     SVC(kernel="linear", C=0.025) - Fail
#     SVC(gamma=2, C=1) - Fail
#     GaussianProcessClassifier(1.0 * RBF(1.0)) - Sparse/dense error
#     DecisionTreeClassifier(max_depth=5) - 89
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1) - 74
#     MLPClassifier(alpha=1) - 91
#     AdaBoostClassifier() - 75
#     GaussianNB() - Sparse/dense error
#     QuadraticDiscriminantAnalysis() - Sparse/dense error


In [115]:
t0 = time()
clf.fit(X_train, y_train.tolist())#, sample_weight=weight_list)
t1 = time()
print ("elapsed time is {} seconds for vectorization".format(int(t1-t0)))
# clf = trainNB(X_train=X_train, y_train=y_train)
print (clf)

elapsed time is 3 seconds for vectorization
MultinomialNB(alpha=0.05, class_prior=None, fit_prior=True)


In [116]:
#dtm_test=vectorizer1.transform(X_test)
pred_test = clf.predict(X_test)
compare_df = pd.DataFrame({'true': y_test, 'pred': pred_test})
pd.crosstab(compare_df.true, compare_df.pred)

pred,CERTIFIED,CERTIFIED-WITHDRAWN,DENIED
true,,,
CERTIFIED,534325,20553,33840
CERTIFIED-WITHDRAWN,38788,4763,2007
DENIED,15649,611,4801
WITHDRAWN,17933,808,1453


In [117]:
from sklearn.metrics import accuracy_score
acc_test = accuracy_score(y_test.tolist(), pred_test)
acc_train = accuracy_score(y_train.tolist(), clf.predict(X_train))
print ('train accuracy', acc_train)
print ('test accruacy', acc_test)

train accuracy 0.804606160114
test accruacy 0.805128114032
